# Australian Job Postings

In [2]:
import pandas as pd
import folium
from plotly.offline import iplot
import plotly.figure_factory as ff
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

In [3]:
jobs = pd.read_csv("./jpo-coded.csv", encoding='latin1')
seek = pd.read_csv("./2018seek.csv", encoding='latin1')
locs = pd.read_csv("./Location.csv", encoding='latin1')
sloc = pd.read_csv("./SeekLocs.csv", encoding='latin1')
jobs.columns = [col.replace(' ', '_').lower() for col in jobs.columns]
locs.columns = [col.replace(' ', '_').lower() for col in locs.columns]
seek.columns = [col.replace(' ', '_').lower() for col in seek.columns]
sloc.columns = [col.replace(' ', '_').lower() for col in sloc.columns]

In [4]:
lat = locs.set_index('location').to_dict()['lat']
long = locs.set_index('location').to_dict()['long']
slat = sloc.set_index('location').to_dict()['lat']
slong = sloc.set_index('location').to_dict()['long']

In [5]:
jobs['lat'] = jobs['region'].map(lat).astype(float)
jobs['long'] = jobs['region'].map(long).astype(float)
seek['lat'] = seek['city'].map(slat).astype(float)
seek['long'] = seek['city'].map(slong).astype(float)
jobs = jobs.dropna(axis=0, subset = ['lat', 'long'])
seek = seek.dropna(axis = 0, subset = ['lat', 'long'])

In [6]:
aus_coord = (-28, 133)

In [7]:
m = folium.Map(location=aus_coord, zoom_start=5,tiles='CartoDBPositron')


In [8]:
jobs['date_listed'] = pd.to_datetime(jobs['date_listed'], format='%d/%m/%Y')
seek['post_date'] = pd.to_datetime(seek['post_date'], format = '%Y-%m-%d')

In [9]:
year = jobs.date_listed.dt.strftime('%Y').astype(int)
jobs['year'] = year
month = jobs.date_listed.dt.strftime('%m').astype(int)
jobs['month'] = month
day = jobs.date_listed.dt.strftime('%d').astype(int)
jobs['day'] = day

year = seek.post_date.dt.strftime('%Y').astype(int)
seek['year'] = year
month = seek.post_date.dt.strftime('%m').astype(int)
seek['month'] = month
day = seek.post_date.dt.strftime('%d').astype(int)
seek['day'] = day

In [10]:
jobs_2 = jobs.drop(columns={"anzsco1_code","anzsco2_code","anzsco4_code","anzsco_2dig_title","anzsco_4dig_title","skill_level","job_location","region"})
seek_2 = seek.drop(columns = {"pageurl","crawl_timestamp","company_name","job_description", 'job_board', 'job_type', 'salary_offered','state','city'})

In [11]:
mapdf = pd.DataFrame(columns=['category','title','type','lat','long','year','month','day', 'state'])
jobs_2.rename(columns = { 'job_classification' : 'category',
                        'job_title' : 'title',
                        'date_listed' : 'date'}, inplace = True) 
seek_2.rename(columns = { 'job_title' : 'title',
                        'job_type' : 'type',
                        'geo' : 'state',
                        'post_date' : 'date'}, inplace = True) 

In [12]:
mapdf = pd.concat([jobs_2, seek_2], keys = ['jobs', 'seek'], sort = False)

In [13]:
mapdf.loc[mapdf.title.str.contains("Manag",na=False)|mapdf.title.str.contains("MANAG",na=False)|mapdf.title.str.contains("CEO",na=False)|mapdf.title.str.contains("Executive",na=False)|mapdf.title.str.contains("EXECUTIVE",na=False)|mapdf.title.str.contains("Coordinator",na=False)|mapdf.title.str.contains("Admin",na=False)|mapdf.title.str.contains("ADMIN",na=False)|mapdf.title.str.contains("Business",na=False)|mapdf.title.str.contains("Chief",na=False)|mapdf.title.str.contains("Director",na=False), 'category'] = 'Management'
mapdf.loc[mapdf.title.str.contains("Assistant",na=False), 'category'] = 'Assistant'
mapdf.loc[mapdf.category.str.contains("Admin", na=False)|mapdf.category.str.contains("PROF", na=False)|mapdf.category.str.contains("ADMIN", na=False)|mapdf.category.str.contains("OFFICE", na=False)|mapdf.category.str.contains("Office", na=False)|mapdf.category.str.contains("SEC",na=False), 'category'] = 'Assistant'
mapdf.loc[mapdf.title.str.contains("Specialist",na=False), 'category'] = 'Other'
mapdf.loc[mapdf.title.str.contains("Financial",na=False)|mapdf.title.str.contains("Finance",na=False)|mapdf.title.str.contains("Trade",na=False)|mapdf.title.str.contains("Account",na=False)|mapdf.title.str.contains("Credit",na=False)|mapdf.title.str.contains("Economist",na=False), 'category'] = 'Finance'
mapdf.loc[mapdf.title.str.contains("Engineer",na=False), 'category'] = 'Engineering'
mapdf.loc[mapdf.title.str.contains("Sales",na=False)|mapdf.title.str.contains("Auditor",na=False), 'category'] = 'Sales'
mapdf.loc[mapdf.title.str.contains("Lawyer",na=False), 'category'] = 'Other'
mapdf.loc[mapdf.title.str.contains("Programmer",na=False)|mapdf.title.str.contains("Developer",na=False)|mapdf.title.str.contains("IT",na=False)|mapdf.title.str.contains("Data",na=False), 'category'] = 'IT'
mapdf.loc[mapdf.title.str.contains("Medical",na=False)|mapdf.title.str.contains("Doctor",na=False), 'category'] = 'Medical'
mapdf.loc[mapdf.title.str.contains("Intern",na=False), 'category'] = 'Other'
mapdf.loc[mapdf.title.str.contains("Trade",na=False), 'category'] = 'Trade'
mapdf.loc[mapdf.title.str.contains("Designer",na=False), 'category'] = 'Design'
mapdf.loc[mapdf.category.str.contains("Trade", na=False)|mapdf.category.str.contains("TRADE", na=False)|mapdf.category.str.contains("Service", na=False)|mapdf.category.str.contains("SERVICE", na=False), 'category'] = 'Trade'
mapdf.loc[mapdf.category.str.contains("Gov", na=False)|mapdf.category.str.contains("GOV", na=False)|mapdf.category.str.contains("INTEL", na=False)|mapdf.category.str.contains("EMERG", na=False)|mapdf.category.str.contains("ANAL", na=False)|mapdf.category.str.contains("DEFEN", na=False)|mapdf.category.str.contains("POLICE", na=False), 'category'] = 'Government'
mapdf.loc[mapdf.category.str.contains("Transport", na=False)|mapdf.category.str.contains("Drive", na=False)|mapdf.category.str.contains("DRIV", na=False)|mapdf.category.str.contains("TRANSPORT", na=False)|mapdf.category.str.contains("DRIVE",na=False)|mapdf.category.str.contains("DELIVER",na=False)|mapdf.category.str.contains("VEHICLE",na=False), 'category'] = 'Distribution'
mapdf.loc[mapdf.category.str.contains("ICT", na=False)|mapdf.category.str.contains("IT", na=False)|mapdf.category.str.contains("PROGRAM",na=False), 'category'] = 'IT'
mapdf.loc[mapdf.category.str.contains("FOOD", na=False)|mapdf.category.str.contains("CAFE", na=False)|mapdf.category.str.contains("RESTAURANT", na=False), 'category'] = 'Food'
mapdf.loc[mapdf.category.str.contains("RETAIL", na=False)|mapdf.category.str.contains("FASHION", na=False), 'category'] = 'Sales'
mapdf.loc[mapdf.category.str.contains("FINANC", na=False)|mapdf.category.str.contains("ECO",na=False) |mapdf.category.str.contains("STAT", na=False)|mapdf.category.str.contains("ACCOUNT", na=False), 'category'] = 'Finance'
mapdf.loc[mapdf.category.str.contains("MANAG", na=False)|mapdf.category.str.contains("Manag", na=False), 'category'] = 'Management'
mapdf.loc[mapdf.category.str.contains("SALE", na=False)|mapdf.category.str.contains("ADVERT", na=False)|mapdf.category.str.contains("Market", na=False), 'category'] = 'Sales'
mapdf.loc[mapdf.category.str.contains("ENGINE", na=False)|mapdf.category.str.contains("TECH", na=False)|mapdf.category.str.contains("Tech", na=False), 'category'] = 'Engineering'
mapdf.loc[mapdf.category.str.contains("SCIEN", na=False)|mapdf.category.str.contains("Sci", na=False), 'category'] = 'Science'
mapdf.loc[mapdf.category.str.contains("FARM", na=False)|mapdf.category.str.contains("Farm", na=False)|mapdf.category.str.contains("GARD", na=False)|mapdf.category.str.contains("FISH", na=False)|mapdf.category.str.contains("ANI",na=False), 'category'] = 'Farming'
mapdf.loc[mapdf.category.str.contains("OPERAT", na=False)|mapdf.category.str.contains("LAB", na=False)|mapdf.category.str.contains("ASS", na=False)|mapdf.category.str.contains("FACT",na=False)|mapdf.category.str.contains("BUILD",na=False)|mapdf.category.str.contains("PLUM", na=False)|mapdf.category.str.contains("ELEC", na=False)|mapdf.category.str.contains("CARP",na=False)|mapdf.category.str.contains("HAND",na=False), 'category'] = 'Trade'
mapdf.loc[mapdf.category.str.contains("SCHOOL", na=False)|mapdf.category.str.contains("LIB", na=False)|mapdf.category.str.contains("EDU", na=False)|mapdf.category.str.contains("Edu", na=False)|mapdf.category.str.contains("TUT",na=False)|mapdf.category.str.contains("TEACH",na=False)|mapdf.category.str.contains("COACH", na=False)|mapdf.category.str.contains("TRAI", na=False), 'category'] = 'Education'
mapdf.loc[mapdf.category.str.contains("FILM", na=False)|mapdf.category.str.contains("MEDIA", na=False)|mapdf.category.str.contains("Media", na=False)|mapdf.category.str.contains("ILLUS", na=False)|mapdf.category.str.contains("ART", na=False)|mapdf.category.str.contains("PHOTO", na=False)|mapdf.category.str.contains("ACT", na=False)|mapdf.category.str.contains("DANCE", na=False)|mapdf.category.str.contains("MUS", na=False), 'category'] = 'Media'
mapdf.loc[mapdf.category.str.contains("NUR", na=False)|mapdf.category.str.contains("DOC", na=False)|mapdf.category.str.contains("GEO",na=False)|mapdf.category.str.contains("MED", na=False)|mapdf.category.str.contains("MID", na=False)|mapdf.category.str.contains("HEAL", na=False)|mapdf.category.str.contains("SAFE", na=False)|mapdf.category.str.contains("THERA", na=False)|mapdf.category.str.contains("PHAR", na=False), 'category'] = 'Medical'

mapdf.loc[mapdf.title.str.contains("Customer",na=False)|mapdf.title.str.contains("Service",na=False)|mapdf.title.str.contains("Marketing",na=False), 'category'] = 'Service'
mapdf.loc[mapdf.title.str.contains("Drive",na=False)|mapdf.title.str.contains("Distibutor",na=False)|mapdf.title.str.contains("Regional",na=False)|mapdf.title.str.contains("Transportation",na=False), 'category'] = 'Distribution'
mapdf.loc[mapdf.title.str.contains("Designer",na=False)|mapdf.title.str.contains("Media",na=False)|mapdf.title.str.contains("Journalists",na=False)|mapdf.title.str.contains("Journalism",na=False)|mapdf.title.str.contains("Journalist",na=False), 'category'] = 'Media'
mapdf.loc[mapdf.title.str.contains("Farm",na=False)|mapdf.title.str.contains("FARM",na=False)|mapdf.title.str.contains("PLANT",na=False)|mapdf.title.str.contains("Plant",na=False), 'category'] = 'Farming'

In [14]:
mapdf = mapdf.loc[(mapdf['category'] == 'Management')|(mapdf['category'] == 'Assistant')|(mapdf['category'] == 'Finance')|(mapdf['category'] == 'Engineering')|(mapdf['category'] == 'Sales')|(mapdf['category'] == 'Trade')|(mapdf['category'] == 'Depecialistign')|(mapdf['category'] == 'Medical')|(mapdf['category'] == 'Government')|(mapdf['category'] == 'Food')|(mapdf['category'] == 'IT')|(mapdf['category'] == 'Science')|(mapdf['category'] == 'Farming')|(mapdf['category'] == 'Education')|(mapdf['category'] == 'Distribution')|(mapdf['category'] == 'Media')]

In [15]:
def generateBaseMap(default_location=[-28,133], default_zoom_start=20):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=5)
    return base_map

In [16]:
base_map = generateBaseMap()

In [17]:
from folium.plugins import HeatMap
map_copy = mapdf.copy()
map_copy['count'] = 1

## Map of Australia
Use the interactive widgets to select the years and jobs you would like to analyuse then click update to see the distribution of jobs around the country.

In [18]:
b = widgets.IntRangeSlider(
    value=[2009, 2018],
    min=2009,
    max=2018,
    step=1,
    description='Year:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

u = widgets.Button(
    description='Update',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Update graph',
    icon=''
)

c = widgets.SelectMultiple(
    options=mapdf.category.unique(),
    values = mapdf.category.unique(),
    rows=19,
    description='Categories',
    disabled=False
)

mapping = widgets.Output()
graphing = widgets.Output()
scattering = widgets.Output()
def on_button_clicked(btn):
    base_map = generateBaseMap()
    map_year = map_copy[map_copy['year'].isin(b.value)]
    map_year = map_year[map_year['category'].isin(c.value)]
    
    HeatMap(data=map_year[['lat', 'long', 'count']].groupby(['lat', 'long']).sum().reset_index().values.tolist(), radius=15, max_zoom=13).add_to(base_map)
    with mapping:
        clear_output()
        display(base_map)

u.on_click(on_button_clicked)

display(b, c, u, mapping)

IntRangeSlider(value=(2009, 2018), description='Year:', max=2018, min=2009)

SelectMultiple(description='Categories', options=('Management', 'Assistant', 'Farming', 'Trade', 'Sales', 'IT'…

Button(button_style='success', description='Update', style=ButtonStyle(), tooltip='Update graph')

Output()

In [19]:
cats = ['Assistant', 'Distribution', 'Education', 'Engineering', 'Farming', 'Finance', 'Food', 'Government', 'IT', 'Management', 'Media', 'Medical', 'Sales', 'Trade']

## Bar plot for states
Use the widgets to pick states to compare and see comparisons for the number of some of the most popular jobs listed.

In [20]:
c_sa = widgets.Checkbox(
    value=False,
    description='SA',
    disabled=False
)
c_nsw = widgets.Checkbox(
    value=False,
    description='NSW',
    disabled=False
)
c_vic = widgets.Checkbox(
    value=False,
    description='VIC',
    disabled=False
)
c_qld = widgets.Checkbox(
    value=False,
    description='QLD',
    disabled=False
)
c_wa = widgets.Checkbox(
    value=False,
    description='WA',
    disabled=False
)
c_tas = widgets.Checkbox(
    value=False,
    description='TAS',
    disabled=False
)
c_nt = widgets.Checkbox(
    value=False,
    description='NT',
    disabled=False
)

up = widgets.Button(
    description='Update',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Update graph',
    icon=''
)

g_sa  = mapdf[mapdf['state'] == 'SA']
g_nsw = mapdf[mapdf['state'] == 'NSW']
g_vic = mapdf[mapdf['state'] == 'VIC']
g_qld = mapdf[mapdf['state'] == 'QLD']
g_wa = mapdf[mapdf['state'] == 'WA']
g_tas = mapdf[mapdf['state'] == 'TAS']
g_nt = mapdf[mapdf['state'] == 'NT']

g_sa = g_sa.groupby(['category']).count().state
g_nsw = g_nsw.groupby(['category']).count().title
g_vic = g_vic.groupby(['category']).count().state
g_qld = g_qld.groupby(['category']).count().state
g_wa = g_wa.groupby(['category']).count().state
g_tas = g_tas.groupby(['category']).count().state
g_nt = g_nt.groupby(['category']).count().state

def update(btn):
    fig = go.Figure()
    if c_sa.value == True:
        fig.add_trace(go.Bar(name = 'SA', x = cats, y = g_sa))
    if c_nsw.value == True:
        fig.add_trace(go.Bar(name = 'NSW', x = cats, y = g_nsw))
    if c_vic.value == True:
        fig.add_trace(go.Bar(name = 'VIC', x = cats, y = g_vic))
    if c_wa.value == True:
        fig.add_trace(go.Bar(name = 'WA', x = cats, y = g_wa))
    if c_nt.value == True:
        fig.add_trace(go.Bar(name = 'NT', x = cats, y = g_nt))
    if c_qld.value == True:
        fig.add_trace(go.Bar(name = 'QLD', x = cats, y = g_qld))
    if c_tas.value == True:
        fig.add_trace(go.Bar(name = 'TAS', x = cats, y = g_tas))
    fig.update_layout(title = "Number of jobs in different states", barmode='group', xaxis_title = 'Job category', yaxis_title = 'Count of jobs')
    with graphing:
        clear_output()
        fig.show()

up.on_click(update)
    
display(c_sa,c_nsw,c_vic,c_qld,c_wa,c_tas,c_nt, up, graphing)

Checkbox(value=False, description='SA')

Checkbox(value=False, description='NSW')

Checkbox(value=False, description='VIC')

Checkbox(value=False, description='QLD')

Checkbox(value=False, description='WA')

Checkbox(value=False, description='TAS')

Checkbox(value=False, description='NT')

Button(button_style='success', description='Update', style=ButtonStyle(), tooltip='Update graph')

Output()

In [21]:
nyc = pd.read_csv("./nyc-jobs.csv", index_col = ['Job ID'], encoding='latin1')

In [22]:
nyc['Posting Date'] = pd.to_datetime(nyc['Posting Date'], format='%Y-%m-%dT%H:%M:%S.%f')

In [23]:
nyc.columns = [col.replace(' ', '_').lower() for col in nyc.columns]

In [24]:
annual = nyc.loc[~nyc.index.duplicated(keep='first')]

In [25]:
annual = annual[(annual.job_category == 'Engineering, Architecture, & Planning')|(annual.job_category == 'Technology, Data & Innovation')|(annual.job_category == 'Public Safety, Inspections, & Enforcement')|(annual.job_category == 'Legal Affairs')|(annual.job_category == 'Finance, Accounting, & Procurement')|(annual.job_category == 'Constituent Services & Community Programs')]

## Scatter plot of salaries
Use the widgets to pick salaries to compare, then click update to see the spread of salaries in differnt fields over time

In [26]:
c_eng = widgets.Checkbox(
    value=False,
    description='Engineering',
    disabled=False
)
c_ict = widgets.Checkbox(
    value=False,
    description='ICT',
    disabled=False
)
c_public = widgets.Checkbox(
    value=False,
    description='Public',
    disabled=False
)
c_legal = widgets.Checkbox(
    value=False,
    description='Legal',
    disabled=False
)
c_finance = widgets.Checkbox(
    value=False,
    description='Finance',
    disabled=False
)
c_community = widgets.Checkbox(
    value=False,
    description='Community',
    disabled=False
)

new = widgets.Button(
    description='Update',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Update graph',
    icon=''
)

x = annual.posting_date
eng = annual[(annual.job_category == 'Engineering, Architecture, & Planning')].salary_range_from
it = annual[(annual.job_category == 'Technology, Data & Innovation')].salary_range_from
public = annual[(annual.job_category == 'Public Safety, Inspections, & Enforcement')].salary_range_from
legal = annual[(annual.job_category == 'Legal Affairs')].salary_range_from
finance = annual[(annual.job_category == 'Finance, Accounting, & Procurement')].salary_range_from
community = annual[(annual.job_category == 'Constituent Services & Community Programs')].salary_range_from

def update(btn):
    fig = go.Figure()
    if c_eng.value == True:
        fig.add_trace(go.Scatter(
            x=x, y=eng, mode= 'markers', name = "Engineering", marker_color = 'rgba(255,0,0,1)'))
    if c_ict.value == True:
        fig.add_trace(go.Scatter(
            x=x, y=it, mode= 'markers', name = "ICT", marker_color = 'rgba(128,128,0,1)'))
    if c_public.value == True:
        fig.add_trace(go.Scatter(
            x=x, y=public, mode= 'markers', name = "Public", marker_color = 'rgba(0,255,0,1)'))
    if c_legal.value == True:
        fig.add_trace(go.Scatter(
            x=x, y=legal, mode= 'markers', name = "Legal", marker_color = 'rgba(0,0,255,1)'))
    if c_finance.value == True:
        fig.add_trace(go.Scatter(
            x=x, y=finance, mode= 'markers', name = "Finance", marker_color = 'rgba(128,0,128,1)'))
    if c_community.value == True:
        fig.add_trace(go.Scatter(
            x=x, y=community, mode= 'markers', name = "Community", marker_color = 'rgba(0,128,128,1)'))
    fig.update_layout(title = "Compare Salaies in Different Fields", xaxis_title = 'Date of job posting', yaxis_title = 'Salary in $')
    with scattering:
        clear_output()
        fig.show()
        
new.on_click(update)
display(c_eng, c_ict, c_legal, c_public, c_finance, c_community, new, scattering)

Checkbox(value=False, description='Engineering')

Checkbox(value=False, description='ICT')

Checkbox(value=False, description='Legal')

Checkbox(value=False, description='Public')

Checkbox(value=False, description='Finance')

Checkbox(value=False, description='Community')

Button(button_style='success', description='Update', style=ButtonStyle(), tooltip='Update graph')

Output()